## Project Radar

This document contains current thinking around project Radar. 

###### Import Dependencies 

In [38]:
import pandas as pd 
import numpy as np 
import os 
import requests 
import pickle 

### Overview

#### Generic Motivation

We are given *some* transactional data. Transactional data are interesting in nature. They can be studied  to understant certain behavioural profiles of the clients. 

1. #total transactions -- 1807920587
2. #transactions with either Citi as sender or Citi as bene bank. -- 1630665952
        a. Citi as sender -- 629944895
        b. Citi as beneficiary -- 914241689
3. #transactions with both sender and bene as Non-Citi --  177254635

- Citi ----- Citi 
- Citi ----- Non-Citi 
- Non-Citi ----- Citi 
- Non-Citi ----- Non-Citi 

- icggfatn_managed
- l4_txn_coredataset

#### Sql queries

Query to get initial partition of the dataset

#### Sanity Checks

In [19]:
total_transactions = 1807920587
citi_transactions = 1630665952
nonciti_transactions = 177254635 


print('total transactions equals to citi and non-citi transactions: ',total_transactions == citi_transactions + nonciti_transactions)

citi_sender = 629944895
citi_beneficiary = 914241689
citi_sender_beneficiary = 86479368

print('Different types of Citi transactions: ', citi_transactions == (citi_sender + citi_bene + citi_sender_beneficiary))

total transactions equals to citi and non-citi transactions:  True
Different types of Citi transactions:  True


#### Data Analysis

##### Get all the distinct transaction id's 

##### Flow Analysis 

Get all the distinct country pairs and their number of transaction

In [2]:
def getCoordinates(country, url = 'https://maps.googleapis.com/maps/api/geocode/json' ):
    
    params = {'sensor': 'false', 'address': country}
    r = requests.get(url, params=params)
    results = r.json()['results']
    if(len(results) == 0):
            ##retry
            #print('retrying')
            return getCoordinates(country)
    else:
        return results[0]['geometry']['location']

In [3]:
os.getcwd()

'D:\\gfa\\code'

In [4]:
cp = pd.read_csv(os.path.join(os.getcwd(), 'cp.csv'));

In [5]:
cp.head()

,l4_txn_coredataset.send_bank_ctry_code,l4_txn_coredataset.bene_bank_ctry_code,txn_count
0,NaN,NaN,137723
1,NaN,DZ,2262
2,AG,JO,4
3,AM,DO,13
4,AO,BO,10


In [7]:
cp['sender_lat'] = getCoordinates(cp['l4_txn_coredataset.send_bank_ctry_code'])

ProxyError: HTTPSConnectionPool(host='maps.googleapis.com', port=443): Max retries exceeded with url: /maps/api/geocode/json?address=nan&address=nan&address=AG&address=AM&address=AO&address=CG&address=CL&address=CM&address=CW&address=CY&address=DJ&address=DK&address=DZ&address=GA&address=GE&address=HK&address=ID&address=IE&address=IL&address=MM&address=MO&address=MX&address=MZ&address=nan&address=PA&address=PE&address=QA&address=RO&address=RS&address=SG&address=SN&address=TL&address=TR&address=TT&address=TZ&address=US&address=ZA&address=ZM&address=AG&address=CW&address=CY&address=GW&address=GY&address=HK&address=ID&address=IE&address=IN&address=JM&address=JO&address=KR&address=KW&address=KY&address=LR&address=MU&address=MV&address=MY&address=nan&address=PL&address=RO&address=SM&address=TO&address=US&address=UZ&address=ZM&address=CD&address=CF&address=CG&address=CO&address=CR&address=CW&address=ES&address=FR&address=GE&address=HK&address=ID&address=IE&address=IN&address=JM&address=JO&ad

In [13]:
x = set(cp['l4_txn_coredataset.send_bank_ctry_code'])

In [14]:
x.update(cp['l4_txn_coredataset.bene_bank_ctry_code'] )

In [23]:
x = list(x)

In [29]:
del x[0]

In [34]:
x

['MM',
 'RW',
 'ME',
 'KW',
 'TG',
 'VN',
 'KY',
 'TT',
 'MN',
 'ML',
 'GL',
 'MY',
 'PG',
 'CI',
 'CM',
 'KI',
 'MG',
 'DE',
 'SO',
 'ID',
 'CS',
 'CH',
 'AZ',
 'WF',
 'BN',
 'CK',
 'BF',
 'H2',
 'KM',
 'RS',
 'BS',
 'GG',
 'MV',
 'SA',
 'BJ',
 'NR',
 'JM',
 'KP',
 'ES',
 'AT',
 'AU',
 'VU',
 'GT',
 'BW',
 'TF',
 'GA',
 'EC',
 'DO',
 'TP',
 'MC',
 'UA',
 'PN',
 'SV',
 'NZ',
 'CZ',
 'CC',
 'VC',
 'ZA',
 'UZ',
 'VA',
 'HT',
 'AM',
 'EE',
 'NO',
 'AI',
 'BR',
 'PL',
 'NI',
 'LC',
 'AR',
 'TN',
 'MX',
 'BQ',
 'GR',
 'TM',
 'GD',
 'AQ',
 'GS',
 'DJ',
 'CF',
 'JP',
 'LU',
 'UY',
 'NP',
 'GM',
 'BZ',
 'AO',
 'IR',
 'LY',
 'PE',
 'CR',
 'OM',
 'TW',
 'SB',
 'UM',
 'DM',
 'T2',
 'LV',
 'DK',
 'SH',
 'MD',
 'FJ',
 'PA',
 'MQ',
 'ER',
 'PT',
 'SS',
 'LT',
 'BV',
 'BD',
 'BT',
 'ET',
 'NU',
 'PW',
 'SR',
 'LB',
 'TC',
 'HU',
 'TD',
 'AW',
 'FK',
 'FM',
 'BM',
 'MS',
 'IM',
 'XX',
 'NG',
 'AG',
 'GH',
 'CN',
 'GP',
 'TH',
 'RU',
 'XK',
 'BA',
 'IQ',
 'YT',
 'MA',
 'CX',
 'VG',
 'BI',
 'CW',
 'EG',

In [35]:
latlongDict = {}
for country in x:
    latlongDict[country] = getCoordinates(country)

In [36]:
latlongDict

{'AD': {'lat': 42.506285, 'lng': 1.521801},
 'AE': {'lat': 26.2851399, 'lng': -80.29179599999999},
 'AF': {'lat': 33.93911, 'lng': 67.709953},
 'AG': {'lat': 17.060816, 'lng': -61.796428},
 'AI': {'lat': 18.220554, 'lng': -63.06861499999999},
 'AL': {'lat': 32.3182314, 'lng': -86.902298},
 'AM': {'lat': 40.069099, 'lng': 45.038189},
 'AN': {'lat': 49.3004246, 'lng': 10.5719357},
 'AO': {'lat': 47.516231, 'lng': 14.550072},
 'AQ': {'lat': -75.250973, 'lng': -0.071389},
 'AR': {'lat': -38.416097, 'lng': -63.61667199999999},
 'AS': {'lat': -14.270972, 'lng': -170.132217},
 'AT': {'lat': 47.516231, 'lng': 14.550072},
 'AU': {'lat': -25.274398, 'lng': 133.775136},
 'AW': {'lat': 43.5360853, 'lng': -89.45050119999999},
 'AZ': {'lat': 34.0489281, 'lng': -111.0937311},
 'BA': {'lat': 43.915886, 'lng': 17.679076},
 'BB': {'lat': 13.193887, 'lng': -59.543198},
 'BD': {'lat': 23.684994, 'lng': 90.356331},
 'BE': {'lat': 50.503887, 'lng': 4.469936},
 'BF': {'lat': 12.238333, 'lng': -1.561593},
 'B

In [42]:
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [43]:
save_obj(latlongDict, 'countryCoordinates')

In [45]:
latlongDict = load_obj('countryCoordinates')

#### Data Prepration